In [1]:
from __future__ import division

In [2]:
# For cpu only
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
import tensorflow as tf
import tflearn

In [4]:
import numpy as np

In [5]:
from tensorflow.python import debug as tf_debug

In [6]:
sess = tf.Session()
# sess = tf_debug.LocalCLIDebugWrapperSession(sess)
lr = tf.placeholder(tf.float32, shape=[])
optim = tf.train.AdamOptimizer(learning_rate=lr)
# sgd_optim = tf.train.GradientDescentOptimizer(learning_rate=lr)

In [7]:
# sess.add_tensor_filter("has_inf_or_nan", tf_debug.has_inf_or_nan)

In [8]:
from tqdm import tqdm

In [9]:
from bokeh.plotting import figure, output_notebook, show
output_notebook()

Loading BokehJS ...

In [10]:
# import BayesianNN

In [11]:
# bnn = BayesianNN.BayesianFC([4,32,32,1])

In [12]:
import Bayesian_FC_Layer, Bayesian_Net

In [13]:
def bbnet(i, o, mean_init, std_init, prior_mean, prior_std, activation=tf.nn.relu):
    return Bayesian_FC_Layer.Bayesian_FC(i, o, 
                                         qw_mean_initial=mean_init, 
                                         qw_std_initial=std_init, 
                                         qb_std_initial=std_init,
                                         pw_mean=prior_mean,
                                         pw_sigma=prior_std,
                                         pb_mean=prior_mean,
                                         pb_sigma=prior_std,
                                         activation=activation
                                         )

In [14]:
std_init = 0.1
mean_init = 0.1
prior_mean = 0.0
prior_std = 1.0
l1 = bbnet(4, 32, mean_init, std_init, prior_mean, prior_std)
l2 = bbnet(32, 32, mean_init, std_init, prior_mean, prior_std)
l3 = bbnet(32, 1, mean_init, std_init, prior_mean, prior_std, activation=tf.identity)

# def bnet_model(input_tensor, local_reparam_trick=False):
#     net = l1.sample(input_tensor, local_reparam_trick)
#     net = l2.sample(net, local_reparam_trick)
#     net_out = l3.sample(net, local_reparam_trick)
#     return net_out
# l1 = Bayesian_FC_Layer.Bayesian_FC(4, 32, qw_mean_initial=mean_init, qw_std_initial=std_init, qb_std_initial=std_init)
# l2 = Bayesian_FC_Layer.Bayesian_FC(32, 32, qw_mean_initial=mean_init, qw_std_initial=std_init, qb_std_initial=std_init)
# l3 = Bayesian_FC_Layer.Bayesian_FC(32, 32, qw_mean_initial=mean_init, qw_std_initial=std_init, qb_std_initial=std_init)
# l4 = Bayesian_FC_Layer.Bayesian_FC(32, 32, qw_mean_initial=mean_init, qw_std_initial=std_init, qb_std_initial=std_init)
# l5 = Bayesian_FC_Layer.Bayesian_FC(32, 1, activation=tf.identity, qw_mean_initial=mean_init, qw_std_initial=std_init, qb_std_initial=std_init)

In [15]:
from Bayesian_Net import log_gaussian_pdf

In [16]:
def bnet_prob(pred, tar):
    return log_gaussian_pdf(pred, tar, 0.1)

In [17]:
bnet = Bayesian_Net.Bayesian_Net([l1, l2, l3], bnet_prob)

In [18]:
def test_func(x):
    e = np.random.normal(0,0.01)
    return x + 0.3*np.sin(2 *np.pi * (x + e)) + 0.3*np.sin(4 * np.pi*(x + e)) + e

In [19]:
data = [(x, test_func(x)) for x in np.linspace(0,0.5,1000)]

In [20]:
data_input = list(map(lambda x: [x[0]], data))
data_input_poly = list(map(lambda x: [x[0], x[0]**2, x[0]**3, x[0]**4], data))
data_target = list(map(lambda x: [x[1]], data))

In [21]:
def normal_nn(optim):
    data_input = tf.placeholder(tf.float32, shape=[None, 1])
    data_target = tf.placeholder(tf.float32, shape=[None, 1])
    
    poly = tf.concat(1, [data_input, data_input**2, data_input**3, data_input**4])
    net = tflearn.fully_connected(poly, 32, activation="relu")
#     net = tflearn.fully_connected(net, 32, activation="relu")
#     net = tflearn.fully_connected(net, 32, activation="relu")
#     net = tflearn.fully_connected(net, 32, activation="relu")
#     net = tflearn.fully_connected(net, 32, activation="relu")
    
    data_output = tflearn.fully_connected(net, 1, activation="linear")
    
    loss = tf.reduce_mean(tf.square(data_output - data_target))
    op = optim.minimize(loss)
    
    return op, data_input, data_target, loss, data_output

In [22]:
batch_size = 128

In [23]:
epoch_loss = []
nn_epoch_loss = []

In [24]:
epochs = 2000 

In [25]:
test_set = [[x] for x in np.linspace(-1,1,1000)]
test_set_poly = [[x,x**2,x**3,x**4] for x in np.linspace(-1,1,1000)]
test_set_flat = [a[0] for a in test_set]
data_input_flat = [a[0] for a in data_input]
data_target_flat = [a[0] for a in data_target]

In [26]:
op, data_x, data_y, loss, data_output = normal_nn(optim)
sess.run(tf.global_variables_initializer())
for e in tqdm(range(1000 * 2)):
    indices = np.random.randint(low=0, high=1000, size=batch_size)
    input_batch = [data_input[i] for i in indices]
    target_batch = [data_target[i] for i in indices]

    _, l = sess.run([op, loss], feed_dict={data_x: input_batch, data_y:target_batch, lr:0.001})
    nn_epoch_loss.append(l)

100%|██████████| 2000/2000 [00:01<00:00, 1709.92it/s]


In [27]:
nn_preds = sess.run(data_output, feed_dict={data_x: test_set})

In [28]:
# op, data_x, data_y, m_scaling, b_size, loss, kl_loss, data_loss, grads = bnn.update_v2(8, optim)
# sess.run(tf.global_variables_initializer())

In [29]:
bnn_data_input = tf.placeholder(tf.float32, shape=[None, 4])
bnn_data_target = tf.placeholder(tf.float32, shape=[None, 1])
kls = tf.placeholder(tf.float32, shape=[])

def bnet_model(local_reparam_trick=False):
    net = l1.sample(bnn_data_input, local_reparam_trick)
    net = l2.sample(net, local_reparam_trick)
    net_out = l3.sample(net, local_reparam_trick)
    return net_out

bnet_loss, klloss, dataloss = bnet.loss(bnet_model, bnn_data_target,kl_scaling=kls, N=8)
bnet_output = bnet_model(local_reparam_trick=False)

copy_old = bnet.copy_variational_parameters()
kl_div = bnet.kl_new_and_old()

grads_to_apply = optim.compute_gradients(bnet_loss)
# print(grads_to_apply)

clipped_policy_grads = []
for (grad, var) in grads_to_apply:
    if grad is not None:
        clipped_policy_grads.append((tf.clip_by_norm(grad, 10), var))

# clipped_policy_grads = grads_to_apply
grad_op = optim.apply_gradients(clipped_policy_grads)

sess.run(tf.global_variables_initializer())

In [30]:
bnn_lr = 0.001

In [31]:
epoch_loss=[]
kl_loss=[]
data_loss=[]
kl_divs=[]

In [32]:
for e in tqdm(range(epochs*10)):
#     input_batch = data_input_poly
#     target_batch = data_target
    if e > epochs*10*0.5:
        hh = 1000
    else:
        hh = 500
    indices = np.random.randint(low=0, high=hh, size=batch_size)
    input_batch = [data_input_poly[i] for i in indices]
    target_batch = [data_target[i] for i in indices]
    
#     kl = 0.0
    kl = batch_size/1000
#     kl = 1.0
#     kl = (2**(epochs - e - 1))/( (2**epochs) - 1) 

#     kl = 0.0

    _, l, kloss, dloss = sess.run([grad_op, bnet_loss, klloss, dataloss], feed_dict={kls:kl, bnn_data_input: input_batch, bnn_data_target:target_batch, lr:bnn_lr})
    epoch_loss.append(l)
    
    if e % 50 == 0:
        div = sess.run(kl_div)
        sess.run(copy_old)
        kl_divs.append(div)
    
    if e % 5 == 0:
        epoch_loss.append(l) 
        kl_loss.append(kloss)
        data_loss.append(dloss)

100%|██████████| 20000/20000 [01:23<00:00, 240.43it/s]


In [33]:
sess.run(bnet.layers[0].qw_sigma)

array([[ 0.56565845,  0.00602122,  0.54827267,  0.52972615,  0.00579688,
         0.55321991,  0.57187426,  0.57468385,  0.49634716,  0.52717757,
         0.5657981 ,  0.47332805,  0.46186459,  0.00845638,  0.01920019,
         0.52383471,  0.5490942 ,  0.55473322,  0.56300437,  0.16100834,
         0.55350679,  0.53742075,  0.56511009,  0.35809705,  0.54070121,
         0.52539897,  0.56768137,  0.54137921,  0.57451636,  0.32254115,
         0.53760499,  0.55642754],
       [ 0.83046305,  0.02160467,  0.81912529,  0.80889916,  0.01497551,
         0.82972372,  0.83785409,  0.83251977,  0.79754019,  0.806687  ,
         0.83128405,  0.77841306,  0.76474261,  0.03957652,  0.1071284 ,
         0.81114399,  0.81788325,  0.82198596,  0.83448213,  0.38874942,
         0.8232978 ,  0.82471436,  0.82964706,  0.65913826,  0.82627332,
         0.82496095,  0.83358157,  0.81853801,  0.84280646,  0.61604393,
         0.81433052,  0.82468736],
       [ 0.95497298,  0.07099888,  0.94735754,  0.9447

In [34]:
# for e in tqdm(range(epochs)):
# #     indices = np.random.randint(low=0, high=1000, size=batch_size)
# #     input_batch = [data_input_poly[i] for i in indices]
# #     target_batch = [data_target[i] for i in indices]
#     input_batch = data_input_poly
#     target_batch = data_target
    
# #     ms = (2**(epochs - e - 1))/( (2**epochs) - 1) 
# #     ms = 1000/(batch_size)
# #     ms = (1000*epochs)/(batch_size*(e+1))
# #     ms = (1000)/(batch_size + e)
#     ms = 0
# #     ms = 1.0
    
# #     grad_nums = [a for (a,b) in grads]
#     qw_mean, qw_p, qb_mean, qb_p = sess.run([bnn.qw_means[0], bnn.qw_ps[0], bnn.qb_means[0], bnn.qb_ps[0]])
#     _, l, k_loss, d_loss, gg = sess.run([op, loss, kl_loss, data_loss, grads], feed_dict={data_x: input_batch, data_y:target_batch, m_scaling:ms, b_size:batch_size, lr:bnn_lr})
# #     print("----------Epoch {}-----------".format(e))
# #     print("{}\n{}\n".format(qb_mean, qb_p))
# #     print(gg[2:])
# #     print()
# #     print("KL:{}, Data:{}\n".format(k_loss, d_loss))
#     if e % 5 == 0:
#         epoch_loss.append(l)   

In [35]:
# sess.run(bnn.qw_ps[0])

In [36]:
# sess.run(bnn.qw_ps[0])
# grads = tf.gradients(loss, w_vars)
# sess.run(w_vars, feed_dict={data_x: data_input_poly, data_y:target_batch, m_scaling:[0.1]})

In [37]:
# pred_input = tf.placeholder(tf.float32, shape=[None, 4])
# pred_output = bnn.sampled_output(pred_input, 1000)
# pred_input, pred_output = bnn.sample(use_mean=False)
predictions = sess.run(bnet_output, feed_dict={bnn_data_input: test_set_poly})
var_trials = 500
for _ in tqdm(range(var_trials-1)):
    predictions = np.concatenate([predictions, sess.run(bnet_output, feed_dict={bnn_data_input: test_set_poly})], axis=1)

100%|██████████| 499/499 [00:00<00:00, 649.62it/s]


In [38]:
pred_means = np.apply_along_axis(lambda x: np.mean(x), 1, predictions)
pred_means[3]

2.7593167

In [39]:
pred_vars = np.apply_along_axis(lambda x: np.std(x, ddof=1), 1, predictions)

In [40]:
predictions.shape

(1000, 500)

In [41]:
p = figure(width=800, height=800, y_range=(-0.5,1.5), x_range=(-1,1))

p.line(data_input_flat, data_target_flat, line_width=0.5, color=(0,0,255,0.5))

err_xs=[]
err_ys=[]
for x, y, yerr in zip(test_set_flat, pred_means, pred_vars):
    err_xs.append((x, x))
    err_ys.append((y - yerr, y + yerr))

p.multi_line(err_xs, err_ys, color=(255,0,0,0.15))

# p.line([a/100 for a in range(100)], [scale*a/100 for a in range(100)], line_width=3, color="blue")
p.line(test_set_flat, pred_means, line_width=2, color="red")

for i in range(0, 200, 25):
    pp = list(map(lambda x: x[i], predictions))
    p.line(test_set_flat, pp, line_width=0.5, color=(255,0,255,0.2))

# p.line(data_input_flat, data_target_flat, line_width=0.5, color=(0,0,255,0.5))

p.line(test_set_flat, nn_preds.flatten(), line_width=2, color="orange")
# Loss
# p.line(range(epochs), epoch_loss, line_width=2, color="green")

show(p)

In [42]:
loss_p = figure(width=500, height=500)

# Loss
# loss_p.line(range(len(epoch_loss)), epoch_loss, line_width=0.5, color="green")

loss_p.line(range(len(kl_loss)), kl_loss, line_width=0.5, color="orange")

loss_p.line(range(len(data_loss)), data_loss, line_width=0.5, color="purple")

# loss_p.line(range(len(nn_epoch_loss)), nn_epoch_loss, line_width=1, color="green")

show(loss_p)

In [43]:
loss_kl = figure(width=500, height=500)

loss_kl.line(range(len(kl_divs)), kl_divs, line_width=0.5, color="red")

show(loss_kl)